# Info
name
size
units


price
in_stock

size

capacity
waist_high
waist_low

backing
brand
name
notes
retailer
shipping
tapes
units
url

total_price = price + shipping
unit_price = total_price / units
ml_per_unit_price = capacity / unit_price

backing: str = "plastic",
brand: str = "",
capacity: int = 0,
in_stock: str = "Maybe",
name: str = "",
notes: str = "",
price: float = 0.00,
retailer: str = "",
shipping: float = 0.0,
size_label: str = "",
tapes: str = "",
units: int = 0,
url: str = "",
waist_high: int = 0,
waist_low: int = 0


# Imports

In [1]:
import requests
import re
import json
import yaml
from enum import Enum
from bs4 import BeautifulSoup, Tag
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from typing import Tuple

# Products

In [144]:
with open('products.yml', 'r') as file:
	products = yaml.safe_load(file)

# Driver

In [49]:
driver = None

def init_driver():
	global driver
	if driver == None:
		driver = webdriver.Firefox()

def driver_to(url):
	global driver
	init_driver()
	if driver.current_url != url:
		driver.get(url)

def driver_to_reset(url):
	global driver
	init_driver()
	driver.get(url)

# Helpers

In [4]:
def test_fetch(retailer, url):
	results = retailer(url, products[url])
	print(*results, sep='\n')

def get_response(url):
	with requests.get(url) as response:
		if response.status_code != 200:
			print(f'Error: {response.status_code} for {url}')
			return
		return response

def get_soup(url):
	return BeautifulSoup(get_page(url), 'html.parser')

def get_page(url):
	return get_response(url).text

def get_data(url):
	return get_response(url).json()

def await_price_data(old_price_data, get_price_data) -> Tuple[bool, float]:
		timeout = 0
		while timeout < 10:
			timeout += 1
			price_data = get_price_data()
			if price_data != old_price_data:
				return (False, price_data)
			sleep(0.1 * timeout)
		return (True, old_price_data)

def calculate_derived_info(info: dict) -> dict:
    unit_price: float = info['price'] / info['units']
    total_price: float = info['price'] + info['shipping']
    ml_per_unit_price: int = int(info['capacity'] / unit_price)
    return validate_info(info | {
        "ml_per_unit_price": ml_per_unit_price,
        "total_price": total_price,
        "unit_price": unit_price,
    })

def validate_info(info: dict) -> dict:
	if info['backing'] is None:
		print(f'[backing] is None for {info}')
	if info['brand'] is None:
		print(f'[brand] is None for {info}')
	if info['capacity'] is None:
		print(f'[capacity] is None for {info}')
	if info['in_stock'] is None:
		print(f'[in_stock] is None for {info}')
	if info['name'] is None:
		print(f'[name] is None for {info}')
	if info['notes'] is None:
		print(f'[notes] is None for {info}')
	if info['price'] is None:
		print(f'[price] is None for {info}')
	if info['retailer'] is None:
		print(f'[retailer] is None for {info}')
	if info['shipping'] is None:
		print(f'[shipping] is None for {info}')
	if info['size_label'] is None:
		print(f'[size_label] is None for {info}')
	if info['tapes'] is None:
		print(f'[tapes] is None for {info}')
	if info['units'] is None:
		print(f'[units] is None for {info}')
	if info['url'] is None:
		print(f'[url] is None for {info}')
	if info['waist_high'] is None:
		print(f'[waist_high] is None for {info}')
	if info['waist_low'] is None:
		print(f'[waist_low] is None for {info}')
	return info

# ABU

In [5]:
ABU_QUANTITIES = {
    10: 'abu_quantity_1',
    40: 'abu_quantity_2',
    80: 'abu_quantity_3',
}

ABU_SIZES = {
    'xs': 'XSmall',
    's': 'Small',
    's_m': 'SMedium',
    'm': 'Medium',
    'l': 'Large',
    'xl': 'XLarge',
    'xl-plus': 'XXLarge',
}


def abu(url, product) -> list[dict]:
    rows = []
    id = product['id']
    info = product['info']
    sizes = product['sizes']
    soup = get_soup(url)

    variations = get_data(
        f'https://us.abuniverse.com/wp-json/wc/store/products/{id}')['variations']
    for variation in variations:
        attributes = variation['attributes']

        # Skip samples and scented variations
        is_sample = attributes[0]['value'] == 'sample'
        is_scented = attributes[2]['value'] == 'no-scent'
        if is_sample or is_scented:
            continue

        size = ABU_SIZES[attributes[1]['value']]

        # Get stock data from variation json
        variation_id = variation['id']
        variation_data = get_data(
            f'https://us.abuniverse.com/wp-json/wc/store/products/{variation_id}'
        )

        for units, price_ident in ABU_QUANTITIES.items():
            rows.append(calculate_derived_info(info |sizes[size] | {
                'price': float(re.search('\d*\.\d*', soup.find('label', {'for': price_ident}).string).group()),
                'in_stock': 'Yes' if variation_data['is_in_stock'] else 'No',
                'size': size,
                'units': units,
            }))

    return rows


test_fetch(abu, 'https://us.abuniverse.com/product/agz/')


{'backing': 'Cloth', 'brand': 'ABU', 'capacity': 7500, 'name': 'AlphaGatorZ', 'notes': None, 'retailer': 'ABU', 'shipping': 0.0, 'tapes': 4, 'url': 'https://us.abuniverse.com/product/agz/', 'waist_high': 36, 'waist_low': 31, 'price': 43.99, 'in_stock': 'No', 'size': 'Medium', 'units': 10, 'ml_per_unit_price': 1704, 'total_price': 43.99, 'unit_price': 4.399}
{'backing': 'Cloth', 'brand': 'ABU', 'capacity': 7500, 'name': 'AlphaGatorZ', 'notes': None, 'retailer': 'ABU', 'shipping': 0.0, 'tapes': 4, 'url': 'https://us.abuniverse.com/product/agz/', 'waist_high': 36, 'waist_low': 31, 'price': 143.99, 'in_stock': 'No', 'size': 'Medium', 'units': 40, 'ml_per_unit_price': 2083, 'total_price': 143.99, 'unit_price': 3.5997500000000002}
{'backing': 'Cloth', 'brand': 'ABU', 'capacity': 7500, 'name': 'AlphaGatorZ', 'notes': None, 'retailer': 'ABU', 'shipping': 0.0, 'tapes': 4, 'url': 'https://us.abuniverse.com/product/agz/', 'waist_high': 36, 'waist_low': 31, 'price': 279.99, 'in_stock': 'No', 'size

# Amazon

In [7]:
def amazon(url, product) -> list[dict]:
	info = product['info']
	driver_to(url)
	soup = BeautifulSoup(driver.page_source, 'html.parser')

	price: float = float(soup.find('span', {'class': 'apexPriceToPay'}).find('span', {'class': 'a-offscreen'}).string[1:])
	in_stock: str = 'Yes' if soup.find('span', {'class': 'a-color-success'}) else 'No'

	return [calculate_derived_info(info | {
		'price': price,
		'in_stock': in_stock
	})]

test_fetch(amazon, 'https://www.amazon.com/dp/B08GL65JKT')

{'backing': 'Plastic', 'brand': 'ABDL Shop', 'capacity': 5000, 'name': 'Carousel', 'size': 'Large', 'notes': None, 'retailer': 'Amazon', 'shipping': 0.0, 'tapes': 2, 'units': 10, 'url': 'https://www.amazon.com/dp/B08GL65JKT', 'waist_high': 44, 'waist_low': 37, 'price': 39.99, 'in_stock': 'Yes', 'ml_per_unit_price': 1250, 'total_price': 39.99, 'unit_price': 3.999}


# Bambino

In [73]:
BAMBINO_SIZES = {
    'XS': 'XSmall',
    'S': 'Small',
    'M': 'Medium',
    'L': 'Large',
    'XL': 'XLarge',
    'XXL': 'XXLarge',
}

def bambino(url, product) -> list[dict]:
	rows = []
	info = product['info']
	data = get_data(url + '.json')['product']

	for variant in data['variants']:

		# Skip samples
		if variant['option2'].startswith('1 Sample'):
			continue

		size_data = variant['option1']
		size_options = re.search(f"([S|M|L|XL])+\/?([S|M|L|XL]+)?", size_data).groups()
		waist_options = re.search(f"(\d+)\\\"-(\d+)", size_data).groups()

		for size_tag in size_options:
			if size_tag is None:
				continue
			size = BAMBINO_SIZES[size_tag]

			rows.append(calculate_derived_info(info | {
				'price': float(variant['price']),
				# TODO: Add in stock selection for Bambino
                'in_stock': 'Maybe',
                'size': size,
                'waist_high': waist_options[1],
                'waist_low': waist_options[0],
                'units': int(variant['option2'][-2:]),
			}))

	return rows

test_fetch(bambino, 'https://bambinodiapers.com/products/copy-bellissimo-all-over-print-diapers')

{'backing': 'Plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Bellissimo V-2', 'notes': None, 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': 'w', 'url': 'https://bambinodiapers.com/products/copy-bellissimo-all-over-print-diapers', 'price': 30.99, 'in_stock': 'Maybe', 'size': 'Medium', 'waist_high': '40', 'waist_low': '32', 'units': 8, 'ml_per_unit_price': 1290, 'total_price': 30.99, 'unit_price': 3.87375}
{'backing': 'Plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Bellissimo V-2', 'notes': None, 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': 'w', 'url': 'https://bambinodiapers.com/products/copy-bellissimo-all-over-print-diapers', 'price': 119.99, 'in_stock': 'Maybe', 'size': 'Medium', 'waist_high': '40', 'waist_low': '32', 'units': 48, 'ml_per_unit_price': 2000, 'total_price': 119.99, 'unit_price': 2.4997916666666664}
{'backing': 'Plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Bellissimo V-2', 'notes': None, 'retailer': 'Bambino', 'shipping': 0.0, 'tapes

# InControl

In [99]:
INCONTROL_SIZES = {
	'XS (Youth)': 'XSmall',
	'Small': 'Small',
	'Medium': 'Medium',
	'M': 'Medium',
	'Large': 'Large',
	'L': 'Large',
	'X-Large': 'XLarge',
	'XL': 'XLarge',
	'XXL - Bariatric': 'XXLarge'
}

INCONTROL_UNITS = [12, 36]

def incontrol(url, product) -> list[dict]:
	def get_price_data():
		return driver.find_element(By.CLASS_NAME, 'price--withoutTax').text

	info = product['info']
	sizes = product['sizes']
	quantities = product.get('units') or None
	rows = []
	driver_to_reset(url)
	soup = BeautifulSoup(driver.page_source, 'html.parser')

	# Setup buttons
	size_buttons = []
	quantity_buttons = []
	for button in driver.find_elements(By.CLASS_NAME, 'form-option'):
		if button.text.startswith('Sample'):
			continue
		elif button.text.startswith(('Bag', 'Case')):
			quantity_buttons.append(button)
		else:
			size_buttons.append(button) 
	
	for size_button in size_buttons:
		size = INCONTROL_SIZES[size_button.text]

		price_data = get_price_data()
		size_button.click()
		err, _ = await_price_data(price_data, get_price_data)
		if err:
			print(f'Timed out: {size_button.text} {url}')
		
		for quantity_button in quantity_buttons:
			units = 0
			if quantities is None:
				units = int(re.search('\d+', quantity_button.text).group())
			else:
				units = quantities[size][quantity_button.text]

			price_data = get_price_data()
			quantity_button.click()
			err, price_data = await_price_data(price_data, get_price_data)
			if err:
				print(f'Timed out: {quantity_button.text} {url}')
			price = float(re.search('\d+\.\d+', price_data).group())

			# TODO: Add in stock selection for InControl
			in_stock = 'Maybe'

			rows.append(calculate_derived_info(info | sizes[size] | {
				'price': price,
                'in_stock': in_stock,
                'size': size,
                'units': units,
			}))

	return rows

test_fetch(incontrol, 'https://incontroldiapers.com/abena-abri-form-original-plastic-x-plus-night/')

Timed out: Bag https://incontroldiapers.com/abena-abri-form-original-plastic-x-plus-night/
Timed out: L https://incontroldiapers.com/abena-abri-form-original-plastic-x-plus-night/
{'backing': 'Plastic', 'brand': 'Abena', 'notes': None, 'retailer': 'InControl', 'shipping': 0.0, 'tapes': 4, 'url': 'https://incontroldiapers.com/abena-abri-form-original-plastic-x-plus-night/', 'name': 'M4', 'capacity': 3600, 'waist_high': 40, 'waist_low': 28, 'price': 33.99, 'in_stock': 'Maybe', 'size': 'Medium', 'units': 14, 'ml_per_unit_price': 1482, 'total_price': 33.99, 'unit_price': 2.427857142857143}
{'backing': 'Plastic', 'brand': 'Abena', 'notes': None, 'retailer': 'InControl', 'shipping': 0.0, 'tapes': 4, 'url': 'https://incontroldiapers.com/abena-abri-form-original-plastic-x-plus-night/', 'name': 'M4', 'capacity': 3600, 'waist_high': 40, 'waist_low': 28, 'price': 79.99, 'in_stock': 'Maybe', 'size': 'Medium', 'units': 42, 'ml_per_unit_price': 1890, 'total_price': 79.99, 'unit_price': 1.90452380952

# Land Of Genie

In [159]:
# A bit tempermental about automation

def land_of_genie(url, product) -> list[dict]:
	rows = []
	info = product['info']
	sizes = product['sizes']
	data = get_data(url)['product']

	for variant in data['variants']:
		if 'Sample' in variant['option2']:
			continue
			
		size = variant['option1']

		rows.append(calculate_derived_info( info | sizes[size] | {
			'price': float(variant['price']),
			'in_stock': 'Maybe',
			'size': size,
			'units': int(variant['option2'][:1]) * 10
		}))

	return rows

test_fetch(land_of_genie, 'https://landofgenie.com/products/landofgenie-adult-abdl-diapers-overnight-adult-diapers-with-10-pieces-2.json')

{'backing': 'Plastic', 'brand': 'Land Of Genie', 'capacity': 5000, 'name': 'Anime Dinosaur', 'retailer': 'Land Of Genie', 'shipping': 0.0, 'tapes': 4, 'url': 'https://landofgenie.com/products/landofgenie-adult-abdl-diapers-overnight-adult-diapers-with-10-pieces-2', 'waist_high': 38, 'waist_low': 28, 'price': 38.99, 'in_stock': 'Maybe', 'size': 'Medium', 'units': 10, 'ml_per_unit_price': 1282, 'total_price': 38.99, 'unit_price': 3.899}
{'backing': 'Plastic', 'brand': 'Land Of Genie', 'capacity': 5000, 'name': 'Anime Dinosaur', 'retailer': 'Land Of Genie', 'shipping': 0.0, 'tapes': 4, 'url': 'https://landofgenie.com/products/landofgenie-adult-abdl-diapers-overnight-adult-diapers-with-10-pieces-2', 'waist_high': 38, 'waist_low': 28, 'price': 124.4, 'in_stock': 'Maybe', 'size': 'Medium', 'units': 40, 'ml_per_unit_price': 1607, 'total_price': 124.4, 'unit_price': 3.1100000000000003}
{'backing': 'Plastic', 'brand': 'Land Of Genie', 'capacity': 5000, 'name': 'Anime Dinosaur', 'retailer': 'Lan

# LittleForBig

In [24]:
LITTLE_FOR_BIG_SIZES = {
	'M': 'Medium',
	'L': 'Large',
}

def little_for_big(url, product) -> list[dict]:
	rows = []
	info = product['info']
	sizes = product['sizes']
	data = get_data(url)

	for variant in data['variations']:
		size, units = re.search('([M|L])(?:[a-zA-z ]*)?(\d*)', variant['attributes'][0]['value']).groups()
		size = LITTLE_FOR_BIG_SIZES[size]

		variant_data = get_data(
			'https://www.littleforbig.com/wp-json/wc/store/products/{0}'.format(variant['id'])
		)

		price = variant_data['prices']['price']
		price = float('{0}.{1}'.format(price[:-2], price[-2:]))

		rows.append(calculate_derived_info(info | sizes[size] | {
			'price': price,
            'in_stock': variant_data['is_in_stock'],
            'size': size,
            'units': int(units),
		}))

	return rows

test_fetch(little_for_big, 'https://www.littleforbig.com/wp-json/wc/store/products/155255')


{'backing': 'Plastic', 'brand': 'LittleForBig', 'capacity': 5352, 'name': 'Blushing Baby', 'notes': None, 'retailer': 'LittleForBig', 'shipping': 0.0, 'tapes': 4, 'url': 'https://www.littleforbig.com/product/blushing-baby-adult-diapers-10-pieces-packm-l/', 'waist_high': 38, 'waist_low': 28, 'price': 139.99, 'in_stock': True, 'size': 'Medium', 'units': 40, 'ml_per_unit_price': 1529, 'total_price': 139.99, 'unit_price': 3.49975}
{'backing': 'Plastic', 'brand': 'LittleForBig', 'capacity': 5352, 'name': 'Blushing Baby', 'notes': None, 'retailer': 'LittleForBig', 'shipping': 0.0, 'tapes': 4, 'url': 'https://www.littleforbig.com/product/blushing-baby-adult-diapers-10-pieces-packm-l/', 'waist_high': 38, 'waist_low': 28, 'price': 248.99, 'in_stock': True, 'size': 'Medium', 'units': 80, 'ml_per_unit_price': 1719, 'total_price': 248.99, 'unit_price': 3.112375}
{'backing': 'Plastic', 'brand': 'LittleForBig', 'capacity': 5352, 'name': 'Blushing Baby', 'notes': None, 'retailer': 'LittleForBig', 'sh

# MyInnerbaby

In [12]:
MY_INNER_BABY_SIZES = {
	'Medium (M)': 'Medium',
	'Large (L)': 'Large',
}

def my_inner_baby(url, product) -> list[dict]:
	rows = []
	info = product['info']
	sizes = product['sizes']
	data = get_data(url)['product']

	for variant in data['variants']:
		if variant['option2'].endswith('Sample'):
			continue

		size = MY_INNER_BABY_SIZES[variant['option1']]

		rows.append(calculate_derived_info(info | sizes[size] | {
			'price': float(variant['price']),
			# TODO: Add in stock selection for MyInnerBaby
            'in_stock': 'Maybe',
            'size': size,
            'units': int(re.search('\d+', variant['option2']).group()),
		}))
		
	return rows

test_fetch(my_inner_baby, 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper.json')


{'backing': 'Plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': None, 'retailer': 'MyInnerBaby', 'shipping': 0.0, 'tapes': 4, 'url': 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper', 'waist_high': 40, 'waist_low': 32, 'price': 39.95, 'in_stock': 'Maybe', 'size': 'Medium', 'units': 10, 'ml_per_unit_price': 1251, 'total_price': 39.95, 'unit_price': 3.995}
{'backing': 'Plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': None, 'retailer': 'MyInnerBaby', 'shipping': 0.0, 'tapes': 4, 'url': 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper', 'waist_high': 40, 'waist_low': 32, 'price': 109.95, 'in_stock': 'Maybe', 'size': 'Medium', 'units': 40, 'ml_per_unit_price': 1819, 'total_price': 109.95, 'unit_price': 2.7487500000000002}
{'backing': 'Plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': None, 're

# NorthShore

In [14]:
NORTHSHORE_SIZES = {
	'X-Small': 'XSMall',
	'Small': 'Small',
	'Medium': 'Medium',
	'Large': 'Large',
	'X-Large': 'XLarge',
}

def northshore(url, product) -> list[dict]:
	info = product['info']
	driver_to(url)
	soup = BeautifulSoup(driver.page_source, 'html.parser')

	size_info, units_info = list(map(lambda el: el.text, soup.find_all('span', {'class': 'value'})))[0:3:2]
	
	size, waist_low, waist_high = re.search('(.*), (\d+) - (\d+)', size_info).groups()
	size = NORTHSHORE_SIZES[size]
	units = int(re.search('(?:Case|Pack)\/(\d+)', units_info).groups()[0])
	return [calculate_derived_info(info | {
		'price': float(soup.find('span', {'class', 'product-details__price_highlight'}).string[1:]),
		'in_stock': 'Yes' if soup.find('span', {'class', 'icon icon-check'}) else 'No',
		'size': size,
		'waist_high': waist_high,
		'waist_low': waist_low,
		'units': units,
	})]

test_fetch(northshore, 'https://www.northshorecare.com/adult-diapers/adult-diapers-with-tabs/northshore-megamax-tab-style-briefs/northshore-megamax-tab-style-briefs-medium-case40-410s')

{'backing': 'Plastic', 'brand': 'NorthShore', 'capacity': 6500, 'name': 'MEGAMAX', 'notes': None, 'retailer': 'NorthShore', 'shipping': 0.0, 'tapes': 4, 'url': 'https://www.northshorecare.com/adult-diapers/adult-diapers-with-tabs/northshore-megamax-tab-style-briefs/northshore-megamax-tab-style-briefs-medium-case40-410s', 'price': 114.99, 'in_stock': 'Yes', 'size': 'Medium', 'waist_high': '44', 'waist_low': '32', 'units': 40, 'ml_per_unit_price': 2261, 'total_price': 114.99, 'unit_price': 2.8747499999999997}


# Rearz

In [140]:
REARZ_SIZES = {
	'Teen': 'XSmall',
	'XS - Teen': 'XSmall',
	'X-Small': 'XSmall',
	'Small': 'Small',
	'Medium': 'Medium',
	'Large': 'Large',
	'X-Large': 'XLarge',
	'XL': 'XLarge',
	'XXL - Bariatric': 'XXLarge',
	'Bariatric': 'XXLarge',
}

def rearz(url, product) -> list[dict]:
	def get_price_data():
		soup = BeautifulSoup(driver.page_source, 'html.parser')
		return soup.find('span', {'class': 'price price--withoutTax'}).text

	rows = []
	info = product['info']
	sizes = product.get('sizes') or None
	quantities = product.get('units') or None
	driver_to_reset(url)
	

	size_buttons = []
	units_buttons = []
	for button in driver.find_elements(By.CLASS_NAME, 'form-option'):
		if button.get_attribute('for').startswith('st'):
			continue
		elif button.text.strip().startswith('Trial') or button.text.strip().startswith('Sample'):
			continue
		elif button.text.strip() == '':
			continue
		elif button.text.startswith(('Bag', 'Case')):
			units_buttons.append(button)
		else:
			size_buttons.append(button)
	
	for size_button in size_buttons:
		size, waist_low, waist_high = re.search('(?:NEW )?([a-zA-Z-]+) \((\d+\.?\d?)(?:\"|\”) ?- ?(\d+\.?\d?)', size_button.text).groups()
		size = REARZ_SIZES[size]

		price_data = get_price_data()
		size_button.click()
		err, _ = await_price_data(price_data, get_price_data)
		if err:
			print(f'Timed out: {url}')

		for units_button in units_buttons:
			units = 0
			if quantities is None:
				units = int(re.search('\d+', units_button.text).group())
			else:
				units = quantities[size][units_button.text]

			price_data = get_price_data()
			units_button.click()
			err, price_data = await_price_data(price_data, get_price_data)
			if err:
				print(f'Timed out: {url}')

			if sizes is not None:
				info = info | sizes[size]

			rows.append(calculate_derived_info(info | {
				'price': float(re.search('\d+\.\d+', price_data).group()),
				# TODO: add stock for Rearz
                'in_stock': 'Maybe',
                'size': size,
                'waist_high': waist_high,
                'waist_low': waist_low,
                'units': units,
			}))
	
	return rows

test_fetch(rearz, 'https://rearz.ca/mermaid-tales/')

{'backing': 'Plastic', 'brand': 'Rearz', 'capacity': 7000, 'name': 'Mermaid Tales', 'notes': None, 'retailer': 'Rearz', 'shipping': 0.0, 'tapes': 4, 'url': 'https://rearz.ca/mermaid-tales/', 'price': 48.99, 'in_stock': 'Maybe', 'size': 'Medium', 'waist_high': '40', 'waist_low': '30', 'units': 14, 'ml_per_unit_price': 2000, 'total_price': 48.99, 'unit_price': 3.4992857142857146}
{'backing': 'Plastic', 'brand': 'Rearz', 'capacity': 7000, 'name': 'Mermaid Tales', 'notes': None, 'retailer': 'Rearz', 'shipping': 0.0, 'tapes': 4, 'url': 'https://rearz.ca/mermaid-tales/', 'price': 104.99, 'in_stock': 'Maybe', 'size': 'Medium', 'waist_high': '40', 'waist_low': '30', 'units': 42, 'ml_per_unit_price': 2800, 'total_price': 104.99, 'unit_price': 2.499761904761905}
{'backing': 'Plastic', 'brand': 'Rearz', 'capacity': 7000, 'name': 'Mermaid Tales', 'notes': None, 'retailer': 'Rearz', 'shipping': 0.0, 'tapes': 4, 'url': 'https://rearz.ca/mermaid-tales/', 'price': 46.99, 'in_stock': 'Maybe', 'size': '

# Tykables

In [16]:
TYKABLES_SIZES = {
    'Medium': 'Medium',
    'Large': 'Large',
    'XL': 'XLarge',
}

def tykables(url, product) -> list[dict]:
    rows = []
    info = product['info']
    sizes = product['sizes']
    data = get_data(url)['product']

    for variant in data['variants']:
        if variant['option3'] != 'None' or variant['option2'].endswith('Sample'):
            continue

        size = TYKABLES_SIZES[variant['option1']]

        rows.append(calculate_derived_info(info | sizes[size] | {
            'price': float(variant['price']),
            'in_stock': 'Yes' if variant['inventory_quantity'] > 0 else 'No',
            'size': size,
            'units': int(variant['option2'][:2]),
        }))

    return rows

test_fetch(tykables, 'https://tykables.com/products/str8up-pink.json')

{'backing': 'Plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': None, 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': 4, 'url': 'https://tykables.com/products/str8up-pink', 'waist_high': 36, 'waist_low': 28, 'price': 47.0, 'in_stock': 'Yes', 'size': 'Medium', 'units': 10, 'ml_per_unit_price': 1702, 'total_price': 47.0, 'unit_price': 4.7}
{'backing': 'Plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': None, 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': 4, 'url': 'https://tykables.com/products/str8up-pink', 'waist_high': 36, 'waist_low': 28, 'price': 125.0, 'in_stock': 'Yes', 'size': 'Medium', 'units': 40, 'ml_per_unit_price': 2560, 'total_price': 125.0, 'unit_price': 3.125}
{'backing': 'Plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': None, 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': 4, 'url': 'https://tykables.com/products/str8up-pink', 'waist_high': 36, 'waist_low': 28, 'price': 240

# XPMedical

In [18]:
XP_MEDICAL_SIZES = {
    'Small': 'Small',
    'Medium': 'Medium',
    'Large': 'Large',
    'X-Large': 'XLarge',
}


def xp_medical(url, product) -> list[dict]:
    rows = []
    info = product['info']
    sizes = product.get('sizes') or None
    soup = get_soup(url)

    matches = re.findall(
        '(Small|Medium|Large|X-Large)[a-zA-Z\d\-\n]+(\d\d)-(\d\d)[a-zA-Z \n]+(\d+)[a-zA-Z ,\d\/\n]+\$(\d+.\d\d)', soup.find('table', {'class': 'sizeInfo'}).text.strip())

    for match in matches:
        size, waist_low, waist_high, units, price = match
        size = XP_MEDICAL_SIZES[size]

        if sizes is not None:
            info = info | sizes[size]

        row = calculate_derived_info(info | {
            'price': float(price),
            'in_stock': 'Maybe',
            'size': size,
            'waist_high': waist_high,
            'waist_low': waist_low,
            'units': int(units),
        })
        rows.append(row)

    return rows

test_fetch(xp_medical, 'https://www.xpmedical.com/xp-btd-crinklz-config-xp')

{'backing': 'Plastic', 'brand': 'Crinklz', 'name': 'Crinklz Original', 'notes': None, 'retailer': 'XPMedical', 'shipping': 0.0, 'tapes': 4, 'url': 'https://www.xpmedical.com/xp-btd-crinklz-config-xp', 'capacity': 2900, 'price': 109.99, 'in_stock': 'Maybe', 'size': 'Small', 'waist_high': '31', 'waist_low': '21', 'units': 60, 'ml_per_unit_price': 1581, 'total_price': 109.99, 'unit_price': 1.8331666666666666}
{'backing': 'Plastic', 'brand': 'Crinklz', 'name': 'Crinklz Original', 'notes': None, 'retailer': 'XPMedical', 'shipping': 0.0, 'tapes': 4, 'url': 'https://www.xpmedical.com/xp-btd-crinklz-config-xp', 'capacity': 4394, 'price': 109.99, 'in_stock': 'Maybe', 'size': 'Medium', 'waist_high': '43', 'waist_low': '29', 'units': 60, 'ml_per_unit_price': 2396, 'total_price': 109.99, 'unit_price': 1.8331666666666666}
{'backing': 'Plastic', 'brand': 'Crinklz', 'name': 'Crinklz Original', 'notes': None, 'retailer': 'XPMedical', 'shipping': 0.0, 'tapes': 4, 'url': 'https://www.xpmedical.com/xp-bt

# All urls

In [142]:
rows = []

domains = {
	'us.abu': abu,
	'www.amazon': amazon,
	'bambino': bambino,
	'incontrol': incontrol,
	'www.littleforbig': little_for_big,
	'myinnerbaby': my_inner_baby,
	'www.northshore': northshore,
	'rearz': rearz,
	'tykables': tykables,
	'www.xpmedical': xp_medical,
}

start_index = 0
index = 0

def check_routine(url, product):
	for start, routine in domains.items():
		if url.startswith(f'https://{start}'):
			attempts = 0
			while attempts < 3:
				attempts += 1
				try:
					return routine(url, product)
				except Exception as e:
					print('error: ' + str(e))
			
			

for url, product in products.items():
	index += 1
	if index < start_index:
		continue
	print(index, url)
	new_rows = check_routine(url, product)
	rows.extend(new_rows)
print('Done!')

with open('sheet.json', 'w') as file:
	json.dump(rows, file, sort_keys=True, indent=4, separators=(',', ': '))

1 https://bambinodiapers.com/products/x-plus-ultrastretch-all-white-diapers
2 https://bambinodiapers.com/products/copy-bellissimo-all-over-print-diapers
3 https://bambinodiapers.com/products/copy-of-bloomeez-all-over-print-diapers
4 https://bambinodiapers.com/products/new-catstronaut-all-over-print-diapers
5 https://bambinodiapers.com/products/magnifico
6 https://bambinodiapers.com/products/new-classico-all-over-print-diapers
7 https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers
8 https://bambinodiapers.com/products/new-karnevalee-all-over-print-diapers
9 https://bambinodiapers.com/products/new-skooldoodle-all-over-print-diapers
10 https://bambinodiapers.com/products/new-of-bellissimo-landing-zone-print-diapers
11 https://bambinodiapers.com/products/new-landing-zone-print-diapers
12 https://bambinodiapers.com/products/new-cont-raptor-landing-zone-diaper
13 https://bambinodiapers.com/products/red-bearon-landing-zone-diaper-2
14 https://bambinodiapers.com/products/new-

In [41]:
with open('products.yml', 'w') as file:
	yaml.dump(products, file, sort_keys=True, indent=2)